# GM-VAE for SMB

In [1]:
num_epochs=10000
num_components=2  # only the starting number of components

## Import Libraries

In [2]:
import json

import numpy as np
import argparse
import random
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data
from scipy.io import loadmat

import sys
sys.path.append('/home/yangz2/projects/gmvae_and_gmmvae/pytorch/')
sys.path.append('../../1906-pcgml/pcgml-gmmVae-exp/modules/')
import vglc_with_path_encodings

from model.GMVAE import *
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/home/yangz2/projects/modules/')
from log_training_progress import email_training_progress

## Load data

In [3]:
def open_json_as_nparray(json_fname):
    with open(json_fname, 'r') as json_f:
        return np.array(json.load(json_f))

In [4]:
chunks_int = open_json_as_nparray('/home/yangz2/projects/gmvae_and_gmmvae/data/smba_chunks_int.json')
chunks_onehot = np.eye(len(np.unique(chunks_int)))[chunks_int]
chunks_onehot.shape

(2698, 16, 16, 12)

In [5]:
chunks_onehot = chunks_onehot.reshape(
    chunks_onehot.shape[0], 
    chunks_onehot.shape[1] * chunks_onehot.shape[2] * chunks_onehot.shape[3]
)
chunks_onehot.shape

(2698, 3072)

In [6]:
def get_random_labels(num_components, size):
    return np.random.randint(num_components, size=size)

## Input Parameters

In [7]:
num_components = 8

In [12]:
########################################################
## Input Parameters
#########################################################
parser = argparse.ArgumentParser(description='PyTorch Implementation of DGM Clustering')

## Used only in notebooks
parser.add_argument('-f', '--file',
                    help='Path for input file. First line should contain number of lines to search in')

## Dataset
parser.add_argument('--dataset', type=str, choices=['mnist'],
                    default='mnist', help='dataset (default: mnist)')
parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')

## GPU
parser.add_argument('--cuda', type=int, default=1,
                    help='use of cuda (default: 1)')
parser.add_argument('--gpuID', type=int, default=0,
                    help='set gpu id to use (default: 0)')

## Training
parser.add_argument('--epochs', type=int, default=num_epochs,
                    help='number of total epochs to run (default: 200)')
parser.add_argument('--batch_size', default=64, type=int,
                    help='mini-batch size (default: 64)')
parser.add_argument('--batch_size_val', default=200, type=int,
                    help='mini-batch size of validation (default: 200)')
parser.add_argument('--learning_rate', default=1e-3, type=float,
                    help='learning rate (default: 0.001)')
parser.add_argument('--decay_epoch', default=-1, type=int, 
                    help='Reduces the learning rate every decay_epoch')
parser.add_argument('--lr_decay', default=0.5, type=float,
                    help='Learning rate decay for training (default: 0.5)')

## Architecture
parser.add_argument('--num_classes', type=int, default=num_components,
                    help='number of classes (default: 10)')  # edited for this task
parser.add_argument('--gaussian_size', default=64, type=int,
                    help='gaussian size (default: 64)')
parser.add_argument('--input_size', default=3072, type=int,
                    help='input size (default: 784)')  # edited for this task

## Partition parameters
parser.add_argument('--train_proportion', default=1, type=float,
                    help='proportion of examples to consider for training only (default: 1.0)')

## Gumbel parameters
parser.add_argument('--init_temp', default=1.0, type=float,
                    help='Initial temperature used in gumbel-softmax (recommended 0.5-1.0, default:1.0)')
parser.add_argument('--decay_temp', default=1, type=int, 
                    help='Set 1 to decay gumbel temperature at every epoch (default: 1)')
parser.add_argument('--hard_gumbel', default=0, type=int, 
                    help='Set 1 to use the hard version of gumbel-softmax (default: 1)')
parser.add_argument('--min_temp', default=0.5, type=float, 
                    help='Minimum temperature of gumbel-softmax after annealing (default: 0.5)' )
parser.add_argument('--decay_temp_rate', default=0.013862944, type=float,
                    help='Temperature decay rate at every epoch (default: 0.013862944)')

## Loss function parameters
parser.add_argument('--w_gauss', default=2, type=float,
                    help='weight of gaussian loss (default: 1)')
parser.add_argument('--w_categ', default=1, type=float,
                    help='weight of categorical loss (default: 1)')
parser.add_argument('--w_rec', default=1, type=float,
                    help='weight of reconstruction loss (default: 1)')
parser.add_argument('--rec_type', type=str, choices=['bce', 'mse'],
                    default='bce', help='desired reconstruction loss function (default: bce)')

## Others
parser.add_argument('--verbose', default=0, type=int,
                    help='print extra information at every epoch.(default: 0)')

args = parser.parse_args()

Set random seed in case it was specified in the parameters


In [13]:
## Random Seed
SEED = args.seed
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if args.cuda:
    torch.cuda.manual_seed(SEED)

## Data Partition

We split the training data into train and validation according to the *train_proportion* parameter:

In [14]:
def get_train_dl_and_valid_dl(chunks, labels):

    valid_indices = np.random.randint(len(chunks), size=100)
    train_chunks, valid_chunks = chunks, chunks[valid_indices]
    train_labels, valid_labels = labels, labels[valid_indices]

    train_ds = TensorDataset(torch.from_numpy(train_chunks).float(), torch.from_numpy(train_labels).long())
    valid_ds = TensorDataset(torch.from_numpy(valid_chunks).float(), torch.from_numpy(valid_labels).long())

    train_dl = DataLoader(train_ds, batch_size=args.batch_size, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=args.batch_size_val, shuffle=False)
    
    return train_dl, valid_dl

## Train Model

In [15]:
email_training_progress(f'Training session has started!')
for num_components in [30, 40, 50]:
    
    args.num_classes = num_components
    gmvae = GMVAE(args)
    
    labels = get_random_labels(num_components, len(chunks_onehot))
    train_dl, valid_dl = get_train_dl_and_valid_dl(chunks_onehot, labels)
    gmvae.train(train_dl, valid_dl)
    
    torch.save(
        gmvae.network.state_dict(),
        f'../qualitative_clustering_analysis/smba_gmvae_{num_components}_{num_epochs}.pth'
    )
    
    email_training_progress(f'GM-VAE with {num_components} components has finished training!')
email_training_progress('Shutdown VM now!')

(Epoch 1 / 10000) Train_Loss: 461.968; Val_Loss: 231.763   Train_ACC: 6.153; Val_ACC: 9.000   Train_NMI: 3.522; Val_NMI: 8.014
(Epoch 2 / 10000) Train_Loss: 229.141; Val_Loss: 227.796   Train_ACC: 5.337; Val_ACC: 8.000   Train_NMI: 2.010; Val_NMI: 0.000
(Epoch 3 / 10000) Train_Loss: 224.856; Val_Loss: 224.211   Train_ACC: 5.597; Val_ACC: 17.000   Train_NMI: 2.567; Val_NMI: 35.593
(Epoch 4 / 10000) Train_Loss: 220.986; Val_Loss: 215.952   Train_ACC: 5.930; Val_ACC: 11.000   Train_NMI: 3.034; Val_NMI: 19.673
(Epoch 5 / 10000) Train_Loss: 213.601; Val_Loss: 210.098   Train_ACC: 5.115; Val_ACC: 13.000   Train_NMI: 1.939; Val_NMI: 20.850
(Epoch 6 / 10000) Train_Loss: 208.917; Val_Loss: 203.213   Train_ACC: 5.374; Val_ACC: 11.000   Train_NMI: 2.159; Val_NMI: 19.413
(Epoch 7 / 10000) Train_Loss: 205.331; Val_Loss: 199.066   Train_ACC: 5.189; Val_ACC: 11.000   Train_NMI: 1.658; Val_NMI: 20.774
(Epoch 8 / 10000) Train_Loss: 202.423; Val_Loss: 199.549   Train_ACC: 5.671; Val_ACC: 14.000   Train_

(Epoch 65 / 10000) Train_Loss: 126.184; Val_Loss: 128.733   Train_ACC: 6.783; Val_ACC: 27.000   Train_NMI: 5.129; Val_NMI: 51.593
(Epoch 66 / 10000) Train_Loss: 126.803; Val_Loss: 126.521   Train_ACC: 6.968; Val_ACC: 24.000   Train_NMI: 4.868; Val_NMI: 50.970
(Epoch 67 / 10000) Train_Loss: 125.535; Val_Loss: 125.413   Train_ACC: 7.042; Val_ACC: 27.000   Train_NMI: 4.595; Val_NMI: 51.691
(Epoch 68 / 10000) Train_Loss: 124.995; Val_Loss: 123.290   Train_ACC: 7.079; Val_ACC: 29.000   Train_NMI: 5.199; Val_NMI: 52.646
(Epoch 69 / 10000) Train_Loss: 123.928; Val_Loss: 123.981   Train_ACC: 7.376; Val_ACC: 27.000   Train_NMI: 5.386; Val_NMI: 53.632
(Epoch 70 / 10000) Train_Loss: 123.376; Val_Loss: 125.475   Train_ACC: 7.116; Val_ACC: 26.000   Train_NMI: 5.299; Val_NMI: 54.050
(Epoch 71 / 10000) Train_Loss: 123.195; Val_Loss: 123.717   Train_ACC: 7.116; Val_ACC: 30.000   Train_NMI: 4.991; Val_NMI: 52.877
(Epoch 72 / 10000) Train_Loss: 121.633; Val_Loss: 123.780   Train_ACC: 6.857; Val_ACC: 31.

(Epoch 129 / 10000) Train_Loss: 87.647; Val_Loss: 86.964   Train_ACC: 7.079; Val_ACC: 32.000   Train_NMI: 4.905; Val_NMI: 58.370
(Epoch 130 / 10000) Train_Loss: 86.544; Val_Loss: 86.742   Train_ACC: 7.339; Val_ACC: 32.000   Train_NMI: 5.567; Val_NMI: 58.647
(Epoch 131 / 10000) Train_Loss: 87.762; Val_Loss: 89.929   Train_ACC: 7.191; Val_ACC: 30.000   Train_NMI: 5.240; Val_NMI: 57.114
(Epoch 132 / 10000) Train_Loss: 86.548; Val_Loss: 86.806   Train_ACC: 7.524; Val_ACC: 30.000   Train_NMI: 5.297; Val_NMI: 57.482
(Epoch 133 / 10000) Train_Loss: 86.511; Val_Loss: 86.568   Train_ACC: 7.339; Val_ACC: 31.000   Train_NMI: 5.571; Val_NMI: 58.702
(Epoch 134 / 10000) Train_Loss: 85.448; Val_Loss: 87.125   Train_ACC: 7.376; Val_ACC: 33.000   Train_NMI: 5.603; Val_NMI: 59.159
(Epoch 135 / 10000) Train_Loss: 85.430; Val_Loss: 85.945   Train_ACC: 7.376; Val_ACC: 30.000   Train_NMI: 5.727; Val_NMI: 58.464
(Epoch 136 / 10000) Train_Loss: 84.570; Val_Loss: 84.885   Train_ACC: 7.153; Val_ACC: 32.000   Tr

(Epoch 193 / 10000) Train_Loss: 66.260; Val_Loss: 64.029   Train_ACC: 7.302; Val_ACC: 34.000   Train_NMI: 5.365; Val_NMI: 60.470
(Epoch 194 / 10000) Train_Loss: 66.960; Val_Loss: 64.141   Train_ACC: 7.191; Val_ACC: 33.000   Train_NMI: 5.402; Val_NMI: 59.455
(Epoch 195 / 10000) Train_Loss: 65.539; Val_Loss: 63.160   Train_ACC: 7.376; Val_ACC: 33.000   Train_NMI: 5.709; Val_NMI: 60.150
(Epoch 196 / 10000) Train_Loss: 65.439; Val_Loss: 65.293   Train_ACC: 7.339; Val_ACC: 34.000   Train_NMI: 5.337; Val_NMI: 59.871
(Epoch 197 / 10000) Train_Loss: 64.414; Val_Loss: 64.045   Train_ACC: 7.228; Val_ACC: 33.000   Train_NMI: 5.215; Val_NMI: 59.660
(Epoch 198 / 10000) Train_Loss: 65.477; Val_Loss: 62.224   Train_ACC: 7.079; Val_ACC: 33.000   Train_NMI: 5.242; Val_NMI: 59.905
(Epoch 199 / 10000) Train_Loss: 65.316; Val_Loss: 66.196   Train_ACC: 7.265; Val_ACC: 32.000   Train_NMI: 5.458; Val_NMI: 59.589
(Epoch 200 / 10000) Train_Loss: 64.724; Val_Loss: 62.813   Train_ACC: 7.376; Val_ACC: 31.000   Tr

(Epoch 257 / 10000) Train_Loss: 54.404; Val_Loss: 54.622   Train_ACC: 7.413; Val_ACC: 33.000   Train_NMI: 5.541; Val_NMI: 59.589
(Epoch 258 / 10000) Train_Loss: 58.984; Val_Loss: 54.887   Train_ACC: 7.339; Val_ACC: 31.000   Train_NMI: 5.296; Val_NMI: 58.938
(Epoch 259 / 10000) Train_Loss: 57.291; Val_Loss: 58.423   Train_ACC: 7.153; Val_ACC: 33.000   Train_NMI: 5.369; Val_NMI: 59.509
(Epoch 260 / 10000) Train_Loss: 54.230; Val_Loss: 53.612   Train_ACC: 7.339; Val_ACC: 32.000   Train_NMI: 5.525; Val_NMI: 58.841
(Epoch 261 / 10000) Train_Loss: 55.719; Val_Loss: 52.384   Train_ACC: 7.376; Val_ACC: 32.000   Train_NMI: 5.364; Val_NMI: 59.523
(Epoch 262 / 10000) Train_Loss: 53.865; Val_Loss: 52.028   Train_ACC: 7.079; Val_ACC: 33.000   Train_NMI: 5.315; Val_NMI: 59.387
(Epoch 263 / 10000) Train_Loss: 54.872; Val_Loss: 52.445   Train_ACC: 7.302; Val_ACC: 33.000   Train_NMI: 5.337; Val_NMI: 59.260
(Epoch 264 / 10000) Train_Loss: 53.620; Val_Loss: 52.661   Train_ACC: 7.302; Val_ACC: 33.000   Tr

...

(Epoch 9692 / 10000) Train_Loss: 21.581; Val_Loss: 24.489   Train_ACC: 6.709; Val_ACC: 38.000   Train_NMI: 13.245; Val_NMI: 72.391
(Epoch 9693 / 10000) Train_Loss: 21.560; Val_Loss: 22.016   Train_ACC: 6.783; Val_ACC: 39.000   Train_NMI: 13.234; Val_NMI: 72.845
(Epoch 9694 / 10000) Train_Loss: 24.543; Val_Loss: 23.464   Train_ACC: 6.746; Val_ACC: 39.000   Train_NMI: 12.938; Val_NMI: 72.096
(Epoch 9695 / 10000) Train_Loss: 22.925; Val_Loss: 20.759   Train_ACC: 6.746; Val_ACC: 40.000   Train_NMI: 12.775; Val_NMI: 72.593
(Epoch 9696 / 10000) Train_Loss: 26.703; Val_Loss: 36.344   Train_ACC: 6.672; Val_ACC: 39.000   Train_NMI: 12.873; Val_NMI: 72.593
(Epoch 9697 / 10000) Train_Loss: 26.120; Val_Loss: 21.417   Train_ACC: 6.857; Val_ACC: 39.000   Train_NMI: 13.065; Val_NMI: 72.362
(Epoch 9698 / 10000) Train_Loss: 26.304; Val_Loss: 23.798   Train_ACC: 6.635; Val_ACC: 41.000   Train_NMI: 12.766; Val_NMI: 73.380
(Epoch 9699 / 10000) Train_Loss: 26.613; Val_Loss: 23.321   Train_ACC: 6.672; Val_A

(Epoch 9755 / 10000) Train_Loss: 22.501; Val_Loss: 23.767   Train_ACC: 6.857; Val_ACC: 41.000   Train_NMI: 13.094; Val_NMI: 73.255
(Epoch 9756 / 10000) Train_Loss: 21.632; Val_Loss: 21.518   Train_ACC: 6.709; Val_ACC: 41.000   Train_NMI: 13.062; Val_NMI: 72.797
(Epoch 9757 / 10000) Train_Loss: 22.864; Val_Loss: 21.472   Train_ACC: 6.820; Val_ACC: 39.000   Train_NMI: 13.160; Val_NMI: 72.460
(Epoch 9758 / 10000) Train_Loss: 23.033; Val_Loss: 24.745   Train_ACC: 6.672; Val_ACC: 40.000   Train_NMI: 12.875; Val_NMI: 72.797
(Epoch 9759 / 10000) Train_Loss: 27.634; Val_Loss: 23.996   Train_ACC: 6.857; Val_ACC: 39.000   Train_NMI: 13.175; Val_NMI: 72.190
(Epoch 9760 / 10000) Train_Loss: 27.896; Val_Loss: 23.380   Train_ACC: 6.746; Val_ACC: 38.000   Train_NMI: 13.206; Val_NMI: 72.096
(Epoch 9761 / 10000) Train_Loss: 23.193; Val_Loss: 21.079   Train_ACC: 6.820; Val_ACC: 39.000   Train_NMI: 13.091; Val_NMI: 73.139
(Epoch 9762 / 10000) Train_Loss: 22.182; Val_Loss: 21.452   Train_ACC: 6.709; Val_A

(Epoch 9818 / 10000) Train_Loss: 22.050; Val_Loss: 20.204   Train_ACC: 6.635; Val_ACC: 42.000   Train_NMI: 13.217; Val_NMI: 73.348
(Epoch 9819 / 10000) Train_Loss: 22.133; Val_Loss: 21.827   Train_ACC: 6.857; Val_ACC: 40.000   Train_NMI: 13.171; Val_NMI: 72.089
(Epoch 9820 / 10000) Train_Loss: 22.269; Val_Loss: 22.492   Train_ACC: 6.857; Val_ACC: 40.000   Train_NMI: 13.108; Val_NMI: 72.797
(Epoch 9821 / 10000) Train_Loss: 21.724; Val_Loss: 21.563   Train_ACC: 6.857; Val_ACC: 41.000   Train_NMI: 13.340; Val_NMI: 72.560
(Epoch 9822 / 10000) Train_Loss: 23.095; Val_Loss: 21.862   Train_ACC: 6.746; Val_ACC: 41.000   Train_NMI: 13.199; Val_NMI: 72.839
(Epoch 9823 / 10000) Train_Loss: 22.077; Val_Loss: 21.113   Train_ACC: 6.783; Val_ACC: 40.000   Train_NMI: 13.080; Val_NMI: 72.585
(Epoch 9824 / 10000) Train_Loss: 22.026; Val_Loss: 22.179   Train_ACC: 6.709; Val_ACC: 39.000   Train_NMI: 13.247; Val_NMI: 73.549
(Epoch 9825 / 10000) Train_Loss: 22.524; Val_Loss: 21.735   Train_ACC: 6.857; Val_A

(Epoch 9881 / 10000) Train_Loss: 22.706; Val_Loss: 23.957   Train_ACC: 6.672; Val_ACC: 39.000   Train_NMI: 12.815; Val_NMI: 71.880
(Epoch 9882 / 10000) Train_Loss: 23.135; Val_Loss: 23.468   Train_ACC: 6.746; Val_ACC: 38.000   Train_NMI: 12.956; Val_NMI: 72.585
(Epoch 9883 / 10000) Train_Loss: 23.338; Val_Loss: 21.903   Train_ACC: 6.894; Val_ACC: 39.000   Train_NMI: 13.166; Val_NMI: 72.824
(Epoch 9884 / 10000) Train_Loss: 23.807; Val_Loss: 22.487   Train_ACC: 6.746; Val_ACC: 41.000   Train_NMI: 13.115; Val_NMI: 72.686
(Epoch 9885 / 10000) Train_Loss: 22.802; Val_Loss: 21.881   Train_ACC: 6.783; Val_ACC: 41.000   Train_NMI: 13.044; Val_NMI: 72.925
(Epoch 9886 / 10000) Train_Loss: 22.489; Val_Loss: 21.548   Train_ACC: 6.783; Val_ACC: 41.000   Train_NMI: 13.091; Val_NMI: 72.686
(Epoch 9887 / 10000) Train_Loss: 25.774; Val_Loss: 21.931   Train_ACC: 6.857; Val_ACC: 41.000   Train_NMI: 13.088; Val_NMI: 73.135
(Epoch 9888 / 10000) Train_Loss: 23.239; Val_Loss: 21.783   Train_ACC: 6.672; Val_A

(Epoch 9944 / 10000) Train_Loss: 21.933; Val_Loss: 20.226   Train_ACC: 6.894; Val_ACC: 40.000   Train_NMI: 13.165; Val_NMI: 72.246
(Epoch 9945 / 10000) Train_Loss: 21.549; Val_Loss: 20.628   Train_ACC: 6.857; Val_ACC: 40.000   Train_NMI: 13.029; Val_NMI: 71.809
(Epoch 9946 / 10000) Train_Loss: 22.426; Val_Loss: 27.130   Train_ACC: 6.746; Val_ACC: 41.000   Train_NMI: 13.047; Val_NMI: 72.739
(Epoch 9947 / 10000) Train_Loss: 28.695; Val_Loss: 27.525   Train_ACC: 6.931; Val_ACC: 39.000   Train_NMI: 13.299; Val_NMI: 71.934
(Epoch 9948 / 10000) Train_Loss: 24.664; Val_Loss: 22.400   Train_ACC: 6.746; Val_ACC: 39.000   Train_NMI: 13.075; Val_NMI: 72.352
(Epoch 9949 / 10000) Train_Loss: 22.267; Val_Loss: 20.257   Train_ACC: 6.709; Val_ACC: 40.000   Train_NMI: 13.117; Val_NMI: 73.211
(Epoch 9950 / 10000) Train_Loss: 22.016; Val_Loss: 21.441   Train_ACC: 6.709; Val_ACC: 39.000   Train_NMI: 13.195; Val_NMI: 72.839
(Epoch 9951 / 10000) Train_Loss: 21.666; Val_Loss: 21.230   Train_ACC: 6.635; Val_A